# Data Plotting Functions

Given the axes, algorithm, epoch, batch_size, lr, and nnsize, return a graph. Unless given a 'title=' the default title is simulated by given parameters, and if that doesn't work then it's 'ABC'. To have labels at the locations of convergence with a text box near it describing the (x,y) coords, you can add conv_label=True. The graph will have a legend [Agent0, Agent1]. The data type accepted are either returns or loss. The y-label and x-labels are automatically simulated. The plot is using seaborn.

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from collections import Counter

%matplotlib inline

sns.set()

### Helper Functions

In [6]:
# Get DataFrame from csv files
def df_data(algo, epoch, batch_size, lr, nnsize):
    fname = algo + "-smallsoccer-0_epochs" + str(epoch) + "_batch_size" + str(batch_size) + "_lr" + str(lr) + "_hidden_sizes" + nnsize + "_.csv"
    file_name = "C:/Users/drago/OneDrive - UW/Ratliff Research/soccergrid_data/smallsoccer0-gridsearch"+ fname
    simple_data = pd.read_csv(file_name)
    return simple_data

# Find mode when converging returns, and return x,y coord
def converge_find(data):
    y = Counter(data).most_common(1)[0][0]
    for i in np.arange(0,len(data)):
        ytest = data[i]
        if (y == ytest):
            x = i
            break
    return x,y

# Given what is graphed return an appropriate xlabel (for now only epoch?)
def get_xlabel():
    xlabel = 'Epochs'
    return xlabel

# Given what is graphed return an appropriate ylabel
def get_ylabel(y_df):
    columns = y_df.columns
    if ('avg_rets' in y_df.columns[0]):
        ylabel = 'Average Returns'
    elif ('loss' in y_df.columns[0]):
        ylabel = 'Loss'
    elif ('grad_norms' in y_df.columns[0]):
        ylabel = 'Gradient Norms'
    else:
        ylabel = 'Not Found'
    return ylabel

# Given what is graphed return an appropriate title
def get_title(y_df, algo, epoch, batch_size, lr, nnsize):
    title = algo + ' agent0 vs agent1 ' + get_ylabel(y_df) + ' (ep =' + str(epoch) + ' bs = ' + str(batch_size) + ' lr = ' + str(lr) + 'nnsize = ' + nnsize + ')'
    return title
    
# Given string list of gradient norms, return float list of grad_norms and RSS for grads         
def grad_norms_to_float(grad_array):
    grad_array = grad_array.strip('][').split(', ')
    for i in np.arange(0,len(grad_array)):
        grad_array[i] = float(grad_array[i])
    root_sum_square = np.sqrt(sum([i**2 for i in grad_array]))
    return grad_array, root_sum_square

### Main Plotting Function

In [7]:
# Given parameters plot graph
def data_plot(fig, ax, algo, epoch, batch_size, lr, nnsize, data,  title='ABC', conv_label=False, figsize=[9, 4]):
    simple_data = df_data(algo, epoch, batch_size, lr, nnsize)
    
    # Plot
    if ('returns' in data):
        data1 = 'agent0_avg_rets'
        data2 = 'agent1_avg_rets'
    elif ('loss' in data):
        data1 = 'agent0_loss'
        data2 = 'agent1_loss'
    y_df = simple_data[[data1, data2]]
    y = df.to_numpy(y_df)
    x = np.arange(0,len(y))
    fig.set_size_inches(figsize)
    ax.plot(x, y[:,0], color='r')
    ax.plot(x, y[:,1], color='b')
    
    # Plot convergence location + label
    if (conv_label):
        x_mode1, y_mode1 = converge_find(y[:,0])
        x_mode2, y_mode2 = converge_find(y[:,1])
        if (x_mode1 !=0):
            ax.plot(x_mode1, y_mode1, 'go')
            ax.text(x_mode1 + 8, y_mode2 + 10, 'x = '+ str(x_mode1) + ' y = '
                     + str(round(y_mode1)), horizontalalignment='center',
                     backgroundcolor=(1,1,1,0.7))
        if (x_mode2 != 0):
            ax.plot(x_mode2, y_mode2, 'go')
            ax.text(x_mode2 + 5, y_mode2 - 7, 'x = '+ str(x_mode2) + ' y = '
                     + str(round(y_mode2)), horizontalalignment='center',
                     backgroundcolor=(1,1,1,0.7))
    
    # Set Title, Xlabel, Ylabel, Legend 
    ax.set_xlim([0, epoch])
    ax.set_xlabel(get_xlabel())
    ax.set_ylabel(get_ylabel(y_df))
    if ('ABC' not in title):
        ax.set_title(title)
    else:
        ax.set_title(get_title(y_df, algo, epoch, batch_size, lr, nnsize))
    ax.legend(['Agent 0', 'Agent 1'])
    return 0

### Extra Plotting Functions

In [ ]:
# Plot all of one datatype onto one graph
def all_data_plot(algo, epochs, batch_sizes, lrs, nnsizes, studies, figsize=[9,4]):
    fig, ax = plt.subplots()
    for epoch in epochs:
        for batch in batch_sizes:
            for lr in lrs:
                for nnsize in nnsizes:
                    for study in studies:
                        data_plot(fig, ax, algo, epoch, batch, lr, nnsize, study, figsize=figsize)

# Given the arrays of possible epochs, batch_sizes, lr, studies, return a map of all data
def all_data_map_plot(algos, epochs, batchs_sizes, lrs, nnsizes, studies):
    fig, ax = plt.subplots(2, 2, figsize=figsize, constrained_layout=True) # 2 Algos, 2 Studies (Returns + Losses)
    algo_count = -1
    study_count = -1
    for algo in algos:
        algo_count +=1
        if (study_count == 1):
            study_count = -1
        for study in studies:
            study_count += 1
            for epoch in epochs:
                for batch in batch_sizes:
                    for lr in lrs:
                        for nnsize in nnsizes:
                            data_plot(fig, ax[algo_count,study_count], algo, epoch, batch, lr, nnsize, study, title= algo + ' ' + study)
